In [ ]:
!git clone https://github.com/MayuriSulane/Snack-Analysis.git


Cloning into 'Snack-Analysis'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 69.23 KiB | 437.00 KiB/s, done.


In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession with the excel package
spark = SparkSession.builder \
    .appName("SnackHealthAnalysis") \
    .getOrCreate()

In [ ]:
import pandas as pd

starbucks_nutri_drink = pd.read_excel("/content/Snack-Analysis/starbucks-menu-nutrition-drinks.xlsx")
starbucks_nutri_food = pd.read_excel("/content/Snack-Analysis/starbucks-menu-nutrition-food.xlsx")
starbucks_drink_menu = pd.read_excel("/content/Snack-Analysis/starbucks_drinkMenu_expanded.xlsx")
mcdonalds = pd.read_excel("/content/Snack-Analysis/menu.xlsx")

In [ ]:
starbucks_nutri_drink.describe()

,Unnamed: 0,Calories,Fat (g),Carb. (g),Fiber (g),Protein,Sodium
count,177,177,177,177,177,177,177
unique,154,30,15,34,8,17,25
top,Iced Coffee,-,-,-,-,-,-
freq,2,85,85,85,85,85,85


In [ ]:
starbucks_nutri_food.describe()

,Calories,Fat (g),Carb. (g),Fiber (g),Protein (g)
count,113.000000,113.000000,113.000000,113.000000,113.000000
mean,356.637168,16.353982,41.486726,2.849558,11.469027
std,127.710685,8.297397,15.796764,2.888466,8.463230
min,90.000000,0.000000,5.000000,0.000000,1.000000
25%,280.000000,9.000000,31.000000,1.000000,5.000000
50%,360.000000,17.000000,42.000000,2.000000,8.000000
75%,450.000000,23.000000,53.000000,3.000000,19.000000
max,650.000000,37.000000,80.000000,21.000000,34.000000


In [ ]:
starbucks_drink_menu.describe()

,Calories,Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV)
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,193.871901,1.307025,0.037603,6.363636,128.884298,35.991736,0.805785,32.962810,6.978512,0.098306,0.036488,0.207562,0.074463
std,102.863303,1.640259,0.071377,8.630257,82.303223,20.795186,1.445944,19.730199,4.871659,0.080980,0.144218,0.145423,0.104865
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,120.000000,0.100000,0.000000,0.000000,70.000000,21.000000,0.000000,18.000000,3.000000,0.040000,0.000000,0.100000,0.000000
50%,185.000000,0.500000,0.000000,5.000000,125.000000,34.000000,0.000000,32.000000,6.000000,0.080000,0.000000,0.200000,0.020000
75%,260.000000,2.000000,0.100000,10.000000,170.000000,50.750000,1.000000,43.750000,10.000000,0.150000,0.000000,0.300000,0.100000
max,510.000000,9.000000,0.300000,40.000000,340.000000,90.000000,8.000000,84.000000,20.000000,0.500000,1.000000,0.600000,0.500000


In [ ]:
# Fix the columns with mixed types
columns_to_fix=["Total Fat (g)","Caffeine (mg)"]
for col in columns_to_fix:
  starbucks_drink_menu[col]=pd.to_numeric(starbucks_drink_menu[col],errors="coerce")

In [ ]:
starbucks_nutri_drink_sdf=spark.createDataFrame(starbucks_nutri_drink)
# starbucks1_sdf.show()
starbucks_nutri_food_sdf=spark.createDataFrame(starbucks_nutri_food)
# starbucks2_sdf.show()
starbucks_drink_menu_sdf=spark.createDataFrame(starbucks_drink_menu)
# starbucks3_sdf.show()
mcdonalds_sdf=spark.createDataFrame(mcdonalds)
# mcdonalds_sdf.show()

In [ ]:
starbucks_drink_menu.dtypes

,0
Beverage_category,object
Beverage,object
Beverage_prep,object
Calories,int64
Total Fat (g),float64
Trans Fat (g),float64
Saturated Fat (g),float64
Sodium (mg),int64
Total Carbohydrates (g),int64
Cholesterol (mg),int64


In [ ]:
starbucks_nutri_drink_sdf.printSchema()
starbucks_nutri_food_sdf.printSchema()
starbucks_drink_menu_sdf.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- Calories: string (nullable = true)
 |-- Fat (g): string (nullable = true)
 |-- Carb. (g): string (nullable = true)
 |-- Fiber (g): string (nullable = true)
 |-- Protein: string (nullable = true)
 |-- Sodium: string (nullable = true)

root
 |-- Unnamed: 0: string (nullable = true)
 |-- Calories: long (nullable = true)
 |-- Fat (g): double (nullable = true)
 |-- Carb. (g): long (nullable = true)
 |-- Fiber (g): long (nullable = true)
 |-- Protein (g): long (nullable = true)

root
 |-- Beverage_category: string (nullable = true)
 |-- Beverage: string (nullable = true)
 |-- Beverage_prep: string (nullable = true)
 |-- Calories: long (nullable = true)
 |-- Total Fat (g): double (nullable = true)
 |-- Trans Fat (g): double (nullable = true)
 |-- Saturated Fat (g): double (nullable = true)
 |-- Sodium (mg): long (nullable = true)
 |-- Total Carbohydrates (g): long (nullable = true)
 |-- Cholesterol (mg): long (nullable = true)
 |-- Dietary F

In [ ]:
starbucks_nutri_drink_sdf.columns

['Unnamed: 0',
 'Calories',
 'Fat (g)',
 'Carb. (g)',
 'Fiber (g)',
 'Protein',
 'Sodium']

In [ ]:
from pyspark.sql.functions import col

# ✅ Clean dataset A (all string types, so cast required)

s1 = starbucks_nutri_drink_sdf.select(
    col("Unnamed: 0").cast("string").alias("Food Name"),
    col("Calories").cast("double").alias("Calories"),
    col("Fat (g)").cast("double").alias("Fat (g)"),
    col("`Carb. (g)`").cast("double").alias("Carbs (g)"),
    col("Fiber (g)").cast("double").alias("Fiber (g)"),
    col("Protein").cast("double").alias("Protein (g)"),
    col("Sodium").cast("double").alias("Sodium (mg)")
)
s1.columns


['Food Name',
 'Calories',
 'Fat (g)',
 'Carbs (g)',
 'Fiber (g)',
 'Protein (g)',
 'Sodium (mg)']

In [ ]:
starbucks_nutri_food_sdf.columns

['Unnamed: 0', 'Calories', 'Fat (g)', 'Carb. (g)', 'Fiber (g)', 'Protein (g)']

In [ ]:
# ✅ Clean dataset B (mostly typed correctly, just rename)
s2 = starbucks_nutri_food_sdf.select(
    col("Unnamed: 0").cast("string").alias("Food Name"),
    col("Calories").cast("double").alias("Calories"),
    col("Fat (g)").cast("double").alias("Fat (g)"),
    col("`Carb. (g)`").cast("double").alias("Carbs (g)"),
    col("Fiber (g)").cast("double").alias("Fiber (g)"),
    col("Protein (g)").cast("double").alias("Protein (g)")
)
s2.columns

['Food Name', 'Calories', 'Fat (g)', 'Carbs (g)', 'Fiber (g)', 'Protein (g)']

In [ ]:
starbucks_drink_menu_sdf.columns

['Beverage_category',
 'Beverage',
 'Beverage_prep',
 'Calories',
 'Total Fat (g)',
 'Trans Fat (g)',
 'Saturated Fat (g)',
 'Sodium (mg)',
 'Total Carbohydrates (g)',
 'Cholesterol (mg)',
 'Dietary Fibre (g)',
 'Sugars (g)',
 'Protein (g)',
 'Vitamin A (% DV)',
 'Vitamin C (% DV)',
 'Calcium (% DV)',
 'Iron (% DV)',
 'Caffeine (mg)']

In [ ]:
from pyspark.sql.functions import col, concat_ws

# ✅ Clean dataset C (fully typed, just rename columns to match)
s3 = starbucks_drink_menu_sdf.select(
    concat_ws(" - ", col("Beverage_category"), col("Beverage"), col("Beverage_prep")).alias("Food Name"),
    col("Calories").cast("double").alias("Calories"),
    col("Total Fat (g)").cast("double").alias("Fat (g)"),
    # col("Trans Fat (g)").cast("double").alias("Trans Fat (g)"),
    # col("Saturated Fat (g)").cast("double").alias("Saturated Fat (g)"),
    col("Sodium (mg)").cast("double").alias("Sodium (mg)"),
    col("Total Carbohydrates (g)").cast("double").alias("Carbs (g)"),
    # col("Cholesterol (mg)").cast("int").alias("Cholesterol (mg)"),
    col("Dietary Fibre (g)").cast("double").alias("Fiber (g)"),
    col("Sugars (g)").cast("double").alias("Sugars (g)"), # Include Sugars (g)
    col("Protein (g)").cast("double").alias("Protein (g)"),
    # col("Vitamin A (% DV)").cast("double").alias("Vitamin A (% DV)"),
    # col("Vitamin C (% DV)").cast("double").alias("Vitamin C (% DV)"),
    # col("Calcium (% DV)").cast("double").alias("Calcium (% DV)"),
    # col("Iron (% DV)").cast("double").alias("Iron (% DV)"),
    # col("Caffeine (mg)").cast("int").alias("Caffeine (mg)")
)
s3.columns

['Food Name',
 'Calories',
 'Fat (g)',
 'Sodium (mg)',
 'Carbs (g)',
 'Fiber (g)',
 'Sugars (g)',
 'Protein (g)']

In [ ]:
type(s3)

pyspark.sql.dataframe.DataFrame

In [ ]:
tdf = s1.unionByName(s2, allowMissingColumns=True)
cdf = tdf.unionByName(s3, allowMissingColumns=True)
cdf.show()

+--------------------+--------+-------+---------+---------+-----------+-----------+----------+
|           Food Name|Calories|Fat (g)|Carbs (g)|Fiber (g)|Protein (g)|Sodium (mg)|Sugars (g)|
+--------------------+--------+-------+---------+---------+-----------+-----------+----------+
|Cool Lime Starbuc...|    45.0|    0.0|     11.0|      0.0|        0.0|       10.0|      NULL|
|    Ombré Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|
|          Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|
|Strawberry Acai S...|    80.0|    0.0|     18.0|      1.0|        0.0|       10.0|      NULL|
|Very Berry Hibisc...|    60.0|    0.0|     14.0|      1.0|        0.0|       10.0|      NULL|
|        Violet Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|
|Evolution Fresh™ ...|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|
|Evolution Fresh™ ...|    NULL|   NULL|     NULL| 

In [ ]:
cdf.summary().show()

+-------+--------------------+------------------+-------+-----------------+------------------+-----------------+------------------+------------------+
|summary|           Food Name|          Calories|Fat (g)|        Carbs (g)|         Fiber (g)|      Protein (g)|       Sodium (mg)|        Sugars (g)|
+-------+--------------------+------------------+-------+-----------------+------------------+-----------------+------------------+------------------+
|  count|                 532|               447|    447|              447|               447|              447|               334|               242|
|   mean|                NULL|222.93512304250558|    NaN|85.35570469798658|1.2483221476510067|7.531991051454139|20.568862275449103| 32.96280991735537|
| stddev|                NULL| 134.8597437713097|    NaN|77.74196197910771| 2.126828541483099|6.623485955763608| 43.17684551344025|19.730199411944326|
|    min|        8-Grain Roll|               0.0|    0.0|              0.0|               0.0|

In [ ]:
mcdonalds_sdf.columns

['Category',
 'Item',
 'Serving Size',
 'Calories',
 'Calories from Fat',
 'Total Fat',
 'Total Fat (% Daily Value)',
 'Saturated Fat',
 'Saturated Fat (% Daily Value)',
 'Trans Fat',
 'Cholesterol',
 'Cholesterol (% Daily Value)',
 'Sodium',
 'Sodium (% Daily Value)',
 'Carbohydrates',
 'Carbohydrates (% Daily Value)',
 'Dietary Fiber',
 'Dietary Fiber (% Daily Value)',
 'Sugars',
 'Protein',
 'Vitamin A (% Daily Value)',
 'Vitamin C (% Daily Value)',
 'Calcium (% Daily Value)',
 'Iron (% Daily Value)']

In [ ]:
mcdonalds_sdf.show()

+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+
| Category|                Item|  Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|
+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+------

In [ ]:
m = mcdonalds_sdf.select(
    col("Item").cast("string").alias("Food Name"),
    col("Calories").cast("double").alias("Calories"),
    col("`Total Fat`").cast("double").alias("Fat (g)"),
    col("Carbohydrates").cast("double").alias("Carbs (g)"),
    col("`Dietary Fiber`").cast("double").alias("Fiber (g)"),
    col("Protein").cast("double").alias("Protein (g)"),
    col("Sodium").cast("double").alias("Sodium (mg)"),
    col("Sugars").cast("double").alias("Sugars (g)") # Include Sugars (g)
)
m.columns

['Food Name',
 'Calories',
 'Fat (g)',
 'Carbs (g)',
 'Fiber (g)',
 'Protein (g)',
 'Sodium (mg)',
 'Sugars (g)']

In [ ]:
m.summary().show()

+-------+--------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|summary|           Food Name|         Calories|           Fat (g)|         Carbs (g)|         Fiber (g)|       Protein (g)|      Sodium (mg)|        Sugars (g)|
+-------+--------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|  count|                 260|              260|               260|               260|               260|               260|              260|               260|
|   mean|                NULL|368.2692307692308|14.165384615384616| 47.34615384615385|1.6307692307692307|13.338461538461539|           495.75|29.423076923076923|
| stddev|                NULL|240.2698864914585|14.205998034646498|28.252231685100476|1.5677170847244275|11.426146163812897|577.0263229811335|28.679796713553245|
|    min| 1% Low Fat Milk Ju

In [ ]:
from pyspark.sql.functions import lit
cdf=cdf.withColumn("Source",lit("Starbucks"))
m=m.withColumn("Source",lit("McDonald's"))

cdf.show(5)
m.show(5)

+--------------------+--------+-------+---------+---------+-----------+-----------+----------+---------+
|           Food Name|Calories|Fat (g)|Carbs (g)|Fiber (g)|Protein (g)|Sodium (mg)|Sugars (g)|   Source|
+--------------------+--------+-------+---------+---------+-----------+-----------+----------+---------+
|Cool Lime Starbuc...|    45.0|    0.0|     11.0|      0.0|        0.0|       10.0|      NULL|Starbucks|
|    Ombré Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|Starbucks|
|          Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|Starbucks|
|Strawberry Acai S...|    80.0|    0.0|     18.0|      1.0|        0.0|       10.0|      NULL|Starbucks|
|Very Berry Hibisc...|    60.0|    0.0|     14.0|      1.0|        0.0|       10.0|      NULL|Starbucks|
+--------------------+--------+-------+---------+---------+-----------+-----------+----------+---------+
only showing top 5 rows

+--------------------+--------

In [ ]:
combined_df=cdf.unionByName(m,allowMissingColumns=True)
combined_df.printSchema()

root
 |-- Food Name: string (nullable = true)
 |-- Calories: double (nullable = true)
 |-- Fat (g): double (nullable = true)
 |-- Carbs (g): double (nullable = true)
 |-- Fiber (g): double (nullable = true)
 |-- Protein (g): double (nullable = true)
 |-- Sodium (mg): double (nullable = true)
 |-- Sugars (g): double (nullable = true)
 |-- Source: string (nullable = false)



In [ ]:
combined_df.show()

+--------------------+--------+-------+---------+---------+-----------+-----------+----------+---------+
|           Food Name|Calories|Fat (g)|Carbs (g)|Fiber (g)|Protein (g)|Sodium (mg)|Sugars (g)|   Source|
+--------------------+--------+-------+---------+---------+-----------+-----------+----------+---------+
|Cool Lime Starbuc...|    45.0|    0.0|     11.0|      0.0|        0.0|       10.0|      NULL|Starbucks|
|    Ombré Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|Starbucks|
|          Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|Starbucks|
|Strawberry Acai S...|    80.0|    0.0|     18.0|      1.0|        0.0|       10.0|      NULL|Starbucks|
|Very Berry Hibisc...|    60.0|    0.0|     14.0|      1.0|        0.0|       10.0|      NULL|Starbucks|
|        Violet Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|Starbucks|
|Evolution Fresh™ ...|    NULL|   NULL|     NULL|     N

In [ ]:
from pyspark.sql.functions import when
combined_df=combined_df.withColumn(
    "Category",
    when(combined_df["Source"]=="McDonald's",
         when(combined_df["Food Name"].contains("Shake") |combined_df["Food Name"].contains("Smoothie")|combined_df["Food Name"].contains("Coffee")|combined_df["Food Name"].contains("Tea")|combined_df["Food Name"].contains("Latte"),"Drink")
         .otherwise("Food")
         ).otherwise(
             when(combined_df["Food name"].contains("Beverage")|combined_df["Food Name"].contains("Coffee")|combined_df["Food Name"].contains("Tea")|combined_df["Food Name"].contains("Latte")|combined_df["Food Name"].contains("Frappuccino"),"Drink")
             .otherwise("Food")
         )
)
combined_df.show(5)

+--------------------+--------+-------+---------+---------+-----------+-----------+----------+---------+--------+
|           Food Name|Calories|Fat (g)|Carbs (g)|Fiber (g)|Protein (g)|Sodium (mg)|Sugars (g)|   Source|Category|
+--------------------+--------+-------+---------+---------+-----------+-----------+----------+---------+--------+
|Cool Lime Starbuc...|    45.0|    0.0|     11.0|      0.0|        0.0|       10.0|      NULL|Starbucks|   Drink|
|    Ombré Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|Starbucks|    Food|
|          Pink Drink|    NULL|   NULL|     NULL|     NULL|       NULL|       NULL|      NULL|Starbucks|    Food|
|Strawberry Acai S...|    80.0|    0.0|     18.0|      1.0|        0.0|       10.0|      NULL|Starbucks|   Drink|
|Very Berry Hibisc...|    60.0|    0.0|     14.0|      1.0|        0.0|       10.0|      NULL|Starbucks|   Drink|
+--------------------+--------+-------+---------+---------+-----------+-----------+-----

In [ ]:
from pyspark.sql.functions import avg
healthy_nutritional_summary=combined_df.groupBy("Source","Category").agg(
    avg("Calories").alias("Average Calories"),
    avg("Fat (g)").alias("Average Fat (g)"),
    avg("Carbs (g)").alias("Average Carbs (g)"),
    avg("Fiber (g)").alias("Average Fiber (g)"),
    avg("Protein (g)").alias("Average Protein (g)"),
    avg("Sodium (mg)").alias("Average Sodium (mg)"),
    avg("Sugars (g)").alias("Average Sugars (g)")
)

healthy_nutritional_summary.show()

+----------+--------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+
|    Source|Category|  Average Calories|  Average Fat (g)| Average Carbs (g)| Average Fiber (g)|Average Protein (g)|Average Sodium (mg)|Average Sugars (g)|
+----------+--------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+
| Starbucks|   Drink|175.65346534653466|              NaN|113.43069306930693|0.4207920792079208|  5.227722772277228| 14.522613065326633| 34.79141104294479|
| Starbucks|    Food| 261.9183673469388|9.388163265306122| 62.20816326530612|1.9306122448979592|  9.431836734693878|  29.48148148148148|29.189873417721518|
|McDonald's|    Food| 405.6521739130435|17.32336956521739| 47.30978260869565|1.9619565217391304| 15.668478260869565|  652.9619565217391|24.532608695652176|
|McDonald's|   Drink| 277.7631578947368|6.519736842105263| 47.43

In [ ]:
nutritional_summary_pd=healthy_nutritional_summary.toPandas()

nutritional_summary_pd.to_csv("healthy_nutritional_report.csv",index=False)

print("Spark DataFrame converted to pandas and exported to healthy_nutritional_report.csv")

Spark DataFrame converted to pandas and exported to healthy_nutritional_summary.csv
